In [236]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os

In [237]:
ID_COOKIES_BUTTON = "onetrust-accept-btn-handler"
CLASS_ITEM_SALE = "new-item-box__overlay"
KEY_WORD_LOCATION = "Sverige"
DATA_TESTID_LOCATION_PARENT = "item-details-location"
CLASS_LOCATION_VALUE = "details-list__item-value"
CLASS_NEXT_PAGE = "web_ui__Pagination__next"
DIRECTORY_OUTPUT = "output"
FILENAME_OUTPUT_LOCATION = "location.json"

In [238]:
driver = webdriver.Chrome()

Aller sur le site de vinted

In [239]:
driver.get("https://www.vinted.se/")

Accepter les cookies si nécessaire

In [240]:
try:
    cookies_button = driver.find_element(By.ID, ID_COOKIES_BUTTON)
    cookies_button.click()
except Exception as e:
    # S'il n'y a pas la popup des cookies
    pass 

Sélectionner manuellement les filtres par défaut

On cherche maintenant à récupérer tous les URLS de tous les produits

In [241]:
def extract_urls_page():
    """
    Extract all URLs of products for sale on the opened page.
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all("a", class_=CLASS_ITEM_SALE)
    urls = [item['href'] for item in items]
    return urls

On trie maintenant les urls en fonction de la localisation

In [242]:
urls_location = {}

In [243]:
def sort_urls_by_location():
    global urls_location

    url_product_list = driver.current_url
    urls_all = extract_urls_page()

    for url in urls_all:
        driver.get(url)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        div_location = soup.find("div", {"data-testid": DATA_TESTID_LOCATION_PARENT})

        if div_location:
            location = div_location.find("div", class_=CLASS_LOCATION_VALUE)
            if location:
                if location.text in urls_location:
                    urls_location[location.text].append(url)
                else:
                    urls_location[location.text] = [url]
            else:
                print("Localisation non mentionnée")
        else:
            print("Description localisation manquante")
            print(driver.page_source)

        # driver.back()  
    driver.get(url_product_list) # Revenir en arrière pour continuer la recherche

Parcourir toute les pages

In [244]:
def go_to_next_page():
    # Passer à la page suivante si elle existe
    try:
        next_button = driver.find_element(By.CLASS_NAME, CLASS_NEXT_PAGE)
        next_button.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        return True
    except Exception as e:
        return False

In [245]:
next_page = True
i = 0
while next_page and i < 1:
    i+=1
    sort_urls_by_location()
    next_page = go_to_next_page()

Filtrer les produits en fonction d'une localisation précise

In [246]:
def filter_urls_by_location(location):
    global urls_location
    url_filtered = {}
    for key, value in urls_location.items():
        if location in key:
            url_filtered[key] = value
    return url_filtered
    

In [247]:
filter_urls_by_location(KEY_WORD_LOCATION)

{'Åkersberga, Sverige': ['https://www.vinted.se/items/3470711409-triopack-bhar'],
 'Solna, Sverige': ['https://www.vinted.se/items/3469964052-bh-med-fina-detaljer']}

Sauvegarde des résultats

In [248]:
if not os.path.exists(DIRECTORY_OUTPUT):
    os.makedirs(DIRECTORY_OUTPUT)

In [249]:
file_path = os.path.join(DIRECTORY_OUTPUT, FILENAME_OUTPUT_LOCATION)
with open(file_path, "w") as file:
    json.dump(urls_location, file)

Fermer le navigateur

In [250]:
driver.quit()